<a href="https://colab.research.google.com/github/armiro/Deep-Learning-A-to-Z/blob/master/Convolutional_Neural_Networks/cnn_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 0 - Mounting Drive into the CoLab's working directory

In [0]:
# This needs to be run just at the first time to bring all Google Drive files
# into an accessible path (after mounting, go to the "files" tab on the left 
# and right-click on your file or folder and click on "copy path")

from google.colab import drive
drive.mount('/content/drive')

# Part 1 - Building the CNN

In [0]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.advanced_activations import LeakyReLU

In [0]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), 
                      input_shape=(64, 64, 3), activation='relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=None))

# Adding a second convolutional layer
classifier.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# devising two pooling layers beside each other, accuracy has been increased to some extent

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=32, activation='relu'))
# classifier.add(LeakyReLU(alpha=0.2))

# using 'LeakyReLU' did not end in a significant change

classifier.add(Dense(units=1, activation='sigmoid'))

# changing activation function to 'tanh' had worse result

In [0]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', 
                   metrics=['accuracy'])

# using predefined 'SGD' had worse result, while 'adamax' had the same result 
# as 'adam'

# Part 2 - Fitting the CNN to the images

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, 
                                   zoom_range=0.2, horizontal_flip=True,
                                   rotation_range=45)

# adding rotation_range resulted in a few increase, whereas adding 
# 'vertical_flip' had worsen the result

test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
# run to see where is the current working directory (cwd) and what are the
# folders/files available under this directory (optional)

import os
cwd = os.getcwd()
files = os.listdir(cwd)
print(files)

In [0]:
# change the directory address according to your Drive path

training_set = train_datagen.flow_from_directory('./drive/My Drive/dataset/training_set',
                                                 target_size=(64, 64),
                                                 batch_size=16,
                                                 class_mode='binary')

# increasing 'batch_size' slows down the system, however a slight increase is observed

test_set = test_datagen.flow_from_directory('./drive/My Drive/dataset/test_set',
                                            target_size=(64, 64),
                                            batch_size=16,
                                            class_mode='binary')

In [0]:
classifier.fit_generator(training_set,
                         steps_per_epoch=8000,
                         epochs=25,
                         validation_data=test_set,
                         validation_steps=2000)

# Part 3 - Saving the model as an HDF5 file

In [0]:
classifier.save('trained_model.h5')